In [ ]:
import funweightclust as funweight
import numpy as np
import skfda
from fitAlzheimer import fitAlzheimerFD, test_predict
from scipy import linalg as scil



In this notebook I will be debugging the difference between the R model and Python models.

In [ ]:
data = fitAlzheimerFD()
fdobj = data['fdx']
fdobjy = data['fdy']
clm = data['groupd']

print(fdobj.coefficients)


Below are the parameters needed to run the main function with the same values as in R

In [ ]:
model = "AkjBkQkDk"
modely = "EII"
model=model.upper()

x = fdobj.coefficients
y = fdobjy.coefficients
N = x.shape[0]
p = x.shape[1]
q = y.shape[1]
K = 2
W = skfda.misc.inner_product_matrix(fdobj.basis, fdobj.basis)
W[W<1.e-15] = 0
W_m = scil.cholesky(W)
dety = scil.det(W)
Wlist = {'W': W, 'W_m': W_m, 'dety':dety}
dfstart = 50
dfupdate = 'approx'
dfconstr = 'no'
itermax=200
threshold=0.001
method="cattell"
eps = 1.e-6
init="kmeans"
init_vector = None
mini_nb = [5, 10]
min_individuals = 2
noise_ctrl = 1.e-8
com_dim = None
d_max = 100
d_set = [2, 2, 2, 2]
known = None
kmeans_control = {"n_init": 1, "max_iter":10, "algorithm":'lloyd'}


ones_row = np.ones((1, N))
DATA = fdobj.coefficients.T
intermediate_bigDATA = W@(DATA)
bigDATA = np.vstack((intermediate_bigDATA, ones_row))

In [ ]:
import funweightclust as funweight
models =["AKJBKQKDK","AKJBQKDK", "AKBKQKDK", "ABKQKDK", "AKBQKDK", "ABQKDK"]
modelsys = ["EII", "VII", "EEI", "VEI"]
new_res = funweight.funweightclust(fdobj, fdobjy, K=2, model=models, modely=modelsys, init="kmeans", nb_rep=1, threshold=0.001)

In [ ]:
groups = 10
ari_scores, confusion_matrices = test_predict(groups, 0.001)

for i in range(groups):
    print(i+1, "ARI Score:\t", ari_scores[i], "\nConfusion Matrix:\n", confusion_matrices[i])

In [ ]:


print("d:\t", new_res.d)
print("a:\t", new_res.a.shape, "\n", new_res.a)
print("b:\t", new_res.b.shape, "\n", new_res.b)
print("mu:\t", new_res.mu.shape, "\n", new_res.mu)
print("prop:\t", new_res.prop.shape, "\n", new_res.prop)
print("ev:\t", new_res.ev.shape, "\n", new_res.ev)



In [ ]:
print("Q:\n", new_res.Q)
print("Q1:\n", new_res.Q1)

In [ ]:
print("gam:\t", new_res.gam.shape, "\n", new_res.gam)
print("cov:\t", new_res.covy.shape, "\n", new_res.covy)
print("icov:\t", new_res.icovy.shape, "\n", new_res.icovy)

In [ ]:
print("Log Likelihood:\t", new_res.loglik)
print("All Log Likelihood:\t", new_res.loglik_all)
print("Complexity:\t", new_res.complexity)
print("Converged:\t", new_res.converged)
print("BIC:\t", new_res.bic)
print("ICL:\t", new_res.icl)

In [ ]:
import sklearn.metrics as met

print("Confusion matrix after debugging:\n",met.confusion_matrix(new_res.cl, clm))
print("ARI Score after debugging:\n", met.adjusted_rand_score(new_res.cl, clm))



The discrepancies begin in _T_mypcat_fd1_Uni. The value for the eigenvectors is in a different order than in R, that could be a reason why the outputs are different. As a result the bj matrices in is in a different order as well.

    bj = np.linalg.solve(W_m, np.eye(W_m.shape[0]))@np.ascontiguousarray(np.real(vecteurs_propres))

Identical values but just in a different order

In [ ]:
from fitAdelaideFD import fitAdelaide


data = fitAdelaide()
new_res = skewfun.tfunHDDC(data['fdx'], data['fdy'], K=2, model=models, modely=modelsys, init="kmeans", nb_rep=1, threshold=0.001)


In [ ]:
import sklearn.metrics as met
clm = data['labels']
print("Confusion matrix after debugging:\n",met.confusion_matrix(new_res.cl, clm))
print("ARI Score after debugging:\n", met.adjusted_rand_score(new_res.cl, clm))